In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('/deal_or_no_deal/data/Deal or No Deal Data.xlsx')

In [4]:
df.head()

,Season,Episode,Contestant ID,Data,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Winnings,Notes
0,1,1,1,25.0,0.01,200.0,5.0,400,5000.0,21000.0,NaN,NaN
1,1,1,1,750000.0,1000000.00,100.0,750.0,400000,37000.0,NaN,NaN,NaN
2,1,1,1,50.0,10.00,100000.0,75000.0,53000,NaN,NaN,NaN,NaN
3,1,1,1,1000.0,1.00,200000.0,77000.0,NaN,NaN,NaN,NaN,NaN
4,1,1,1,10000.0,300000.00,63000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from deal_or_no_deal.preprocess import preprocess_historical_case_data

In [9]:
case_data_df = preprocess_historical_case_data(df)

In [10]:
case_data_df.head()

,case_0.01,case_1.0,case_5.0,case_10.0,case_25.0,case_50.0,case_75.0,case_100.0,case_200.0,case_300.0,...,case_300000.0,case_400000.0,case_500000.0,case_750000.0,case_1000000.0,round_got_better,round,expected_value,offer,percentage_difference
0,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,170639.300000,21000.0,0.123067
1,1,0,1,0,1,0,0,1,1,0,...,0,1,0,1,1,1,2,84129.066667,37000.0,0.439800
2,1,0,1,1,1,1,0,1,1,0,...,0,1,0,1,1,1,3,98806.909091,53000.0,0.536400
3,1,1,1,1,1,1,0,1,1,0,...,0,1,0,1,1,1,4,110734.375000,77000.0,0.695358
4,1,1,1,1,1,1,0,1,1,0,...,1,1,0,1,1,1,5,95979.166667,63000.0,0.656392


In [11]:
case_data_df_subset = case_data_df.drop(columns='offer')

In [14]:
from deal_or_no_deal.model import BankerModel

In [15]:
banker_model = BankerModel()

In [44]:
X = case_data_df_subset.drop(columns='percentage_difference').values
y = case_data_df_subset['percentage_difference'].values

In [45]:
banker_model.fit(X, y)

Fitting linear model...
Fitting XGBoost model...


In [46]:
banker_model.save('/deal_or_no_deal/data/banker_model_0906.pkl')

In [47]:
X.shape

(382, 29)

In [48]:
X[0:1]

array([[1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00,
        1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        1.706393e+05]])

In [49]:
import numpy as np

In [50]:
a = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 61464.00052631579])

In [51]:
a.shape

(29,)

In [30]:
banker_model.predict(X[0:1])

array([0.07850086])

In [53]:
import gym

In [54]:
env = gym.make('deal-or-no-deal-v0')

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [72]:
env.reset()

((array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
         1, 0, 0, 0]),
  array([[19576.78742846]]),
  array([1])),
 0,
 False,
 {})

In [73]:
env.step(0)

((array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
         1, 0, 1, 1]),
  array([[16167.38407232]]),
  array([2])),
 0,
 False,
 {})

In [74]:
env.step(1)

((array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]),
  array([0]),
  array([10])),
 -10,
 True,
 {})

In [75]:
env.final_amount

array([16167.38407232])

In [76]:
env.banker_offers

[array([19576.78742846]),
 array([16167.38407232]),
 array([55509.58824016]),
 array([53675.80280316]),
 array([77873.325445]),
 array([89363.63541046]),
 array([78980.60622964]),
 array([137160.54301382])]